In [ ]:
# -*- coding: utf-8 -*-

Load libs

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import math,random,os
import matplotlib.pyplot as plt
import seaborn as sns

Set pd options

In [ ]:
pd.set_option('display.max_columns', None)

Define const values

In [ ]:
exp=2.7182818284590452353602874713526624977572470936999595749669676277240766303535475945713821785251664274274663919320030599218174135966290435729003342952605956307381323286279434907632338298807531952510190115738341879307021540891499348841675092447614606680822648001684774118537423454424371075390777449920695517027618386062613313845830007520449338265602976067371132007093287091274437470472306969772093101416928368190255151086574637721112523897844250569536967707854499699679468644549059879316368892300987931277361782154249992295763514822082698951936680331825288693984964651058209392398294887933203625094431173012381970684161403970198376793206832823764648042953118023287825098194558153017567173613320698112509961818815930416903515988885193458072738667385894228792284998920868058257492796104841984443634632449684875602336248270419786232090021609902353043699418491463140934317381436405462531520961836908887070167683964243781405927145635490613031072085103837505101157477041718986106873969655212671546889570350354
c=	299792458 #M/S
MeV_to_KG=5.60958*(10**29)
MeVs_to_kgms=5.344/(10**22)
electron_mass=0.0005489 #AEM
electron_mass_kg=9.1093837/(10**31) #9.1093837
aem=6.022e+26#1 кг

Set seed

In [ ]:
seed=42
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(seed)

Read data

In [ ]:
train = pd.read_excel('For_model_labled.xlsx')
test = pd.read_excel('For_check_unlabled.xlsx')

Replace and fill M

In [ ]:
def calc_M(phi1,phi2,eta1,eta2,p1,p2):
    return np.sqrt( 2*p1*p2* ( np.cosh(eta1-eta2) - np.cos( phi1 - phi2 ) ) )

In [ ]:
train['M_new']=calc_M(train['phi1'],train['phi2'],train['eta1'],train['eta2'],train['pt1'],train['pt2'])#more accurate
test['M_new']=calc_M(test['phi1'],test['phi2'],test['eta1'],test['eta2'],test['pt1'],test['pt2'])#more accurate
train=train.drop('M',axis=1)
test=test.drop('M',axis=1)

# Some formulas

New p (outdated formula, also wrong)

In [ ]:
#train['p_1']=abs((train['pt1']*(exp**(2*train['eta1']) +1))/(exp**(2*train['eta1'])-1))
#train['p_2']=abs((train['pt2']*(exp**(2*train['eta2']) +1))/(exp**(2*train['eta2'])-1))

#test['p_1']=abs((test['pt1']*(exp**(2*test['eta1']) +1))/(exp**(2*test['eta1'])-1))
#test['p_2']=abs((test['pt2']*(exp**(2*test['eta2']) +1))/(exp**(2*test['eta2'])-1))

New p (outdated formula, also wrong)

In [ ]:
#train['p1']=np.sqrt(((train['E1']*1000)**2)-(electron_mass**2)*(931.5**2))/931.5
#train['p2']=np.sqrt(((train['E2']*1000)**2)-(electron_mass**2)*(931.5**2))/931.5
#test['p1']=np.sqrt(((test['E1']*1000)**2)-(electron_mass**2)*(931.5**2))/931.5
#test['p2']=np.sqrt(((test['E2']*1000)**2)-(electron_mass**2)*(931.5**2))/931.5

Get more accutrate E through px, py, pz. (doesnt improve score for some reason)

In [ ]:
#train['E1']=np.sqrt((train['px1']**2)+(train['py1']**2)+(train['pz1']**2)) #more accurate
#train['E2']=np.sqrt((train['px2']**2)+(train['py2']**2)+(train['pz2']**2)) #more accurate

Funcs to get joules from eV and kg*m/s from MeV/s

In [ ]:
def get_joules(num):
    return 1.6022e-13*num

def get_kgms(num):
    return num*MeVs_to_kgms

Calculate longitudinal momentum

In [ ]:
#train['pl1']=np.sqrt((train['py1']**2)+(train['pz1']**2))
#train['pl2']=np.sqrt((train['py2']**2)+(train['pz2']**2))
#test['pl1']=np.sqrt((test['py1']**2)+(test['pz1']**2))
#test['pl2']=np.sqrt((test['py2']**2)+(test['pz2']**2))

Copy E1, E2

In [ ]:
train['E1_copy']=train['E1']
train['E2_copy']=train['E2']
test['E1_copy']=test['E1']
test['E2_copy']=test['E2']

Calculate energy of both particles

In [ ]:
#train['ttl_E']=train['E1']+train['E2']
#test['ttl_E']=test['E1']+test['E2']

Calculate rest energy

In [ ]:
#train['rest_energy']=(train['M_new']*1000)*(931.5**2)
#test['rest_energy']=(test['M_new']*1000)*(931.5**2)

Function to get particle speed (mv^2/2 formula from relative physics)

In [ ]:
def calc_speed(E):
    return c*(np.sqrt(1- ((electron_mass_kg*(c**2))/(E+electron_mass_kg*(c**2)))**2 ))

Get c-v (where c is light speed and v is particle speed)

In [ ]:
#train['v1']=(c-calc_speed(get_joules(train['E1']*1000)))**2
#train['v2']=(c-calc_speed(get_joules(train['E2']*1000)))**2

#test['v1']=(c-calc_speed(get_joules(test['E1']*1000)))**2
#test['v2']=(c-calc_speed(get_joules(test['E2']*1000)))**2

In [ ]:
#train['w1']=np.arctanh((c-train['v1'])/c)
#train['w2']=np.arctanh((c-train['v2'])/c)
#test['w1']=np.arctanh((c-test['v1'])/c)
#test['w2']=np.arctanh((c-test['v2'])/c)

Func to calculate particle momentum (p=mv formula from relative physics)

In [ ]:
def calc_momenta(v):
    return electron_mass_kg*v/np.sqrt(1-((v**2)/(c**2)))

Get particle momentum in kg*m/s

In [ ]:
#train['momenta1_kgm_per_s']=calc_momenta(c-train['v1'])
#train['momenta2_kgm_per_s']=calc_momenta(c-train['v2'])
#test['momenta1_kgm_per_s']=calc_momenta(c-test['v1'])
#test['momenta2_kgm_per_s']=calc_momenta(c-test['v2'])

Func to calculate particle rapidity

In [ ]:
def calc_rapidity(eta,transverse):
    return eta-(np.tanh(eta)/2)*(electron_mass_kg/transverse)**2

Get particles rapidity

In [ ]:
#train['y1']=calc_rapidity(train['eta1'],get_joules(train['pt1']*1000))
#train['y2']=calc_rapidity(train['eta2'],get_joules(train['pt2']*1000))
#test['y1']=calc_rapidity(test['eta1'],get_joules(test['pt1']*1000))
#test['y2']=calc_rapidity(test['eta2'],get_joules(test['pt2']*1000))

Get diff between 2 particles

In [ ]:
#train['diff_R']=np.sqrt(((train['y1']-train['y2'])**2)+((train['phi1']-train['phi2'])**2))
#test['diff_R']=np.sqrt(((test['y1']-test['y2'])**2)+((test['phi1']-test['phi2'])**2))

Get transverse energy

In [ ]:
#train['Et1']=np.sqrt((train['pt1']**2)+(train['M_new']**2))
#train['Et2']=np.sqrt((train['pt2']**2)+(train['M_new']**2))
#test['Et1']=np.sqrt((test['pt1']**2)+(test['M_new']**2))
#test['Et2']=np.sqrt((test['pt2']**2)+(test['M_new']**2))

Drop features with low importance for catboost

In [ ]:
unimportant=['px1','px2','py1','py2','pz1','pz2','phi1','phi2']#,'w1','w2','y2']

train=train.drop(columns=unimportant)
test=test.drop(columns=unimportant)

Drop features with high correlation

In [ ]:
#high_corr=['y1','eta2','M_new'] #high corr with eta1,y2,rest_energy

#train=train.drop(columns=high_corr)
#test=test.drop(columns=high_corr)

Prepare data for train and predict

In [ ]:
y = train['Q2']
X = train.drop(columns = ['Q2'])

X=X.drop(columns=['Event','Run'])
ev=test['Event']
test=test.drop(columns=['Event','Run'])

# Train CatBoostClassifier on 4 folds and predict test (mean result from 4 folds)
Iterations - can be higher or lower, doesn't matter, it's just for cb to find best iter

Random_state - seeded to get peproducability

Eval_metric - F1, competition metric

Learning_rate - 0.5, very high, but it proved to be best on public leaderboard. My guess is that it's harder for model to overfit this way, while maintaining some kind of learning process without giving up all old patterns.

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
n_splits=4
kf = KFold(n_splits=n_splits)

preds=np.zeros(len(test))
val_scr=0

for train_index, test_index in kf.split(X):
    X_train, X_val = X.values[train_index], X.values[test_index]
    y_train, y_val = y.values[train_index], y.values[test_index]
    rfc = CatBoostClassifier(iterations=100, random_state=seed, eval_metric='F1',learning_rate=0.5)
    rfc.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True,verbose=0)
    print(rfc.best_score_,rfc.best_iteration_)
    val_scr+=rfc.best_score_['validation']['F1']
    pred=rfc.predict(test.values)
    preds+=pred


In [ ]:
val_scr/=n_splits
preds/=n_splits

Round predictions and get them to proper format

In [ ]:
for i,pred in enumerate(preds):
    if pred>0:
        preds[i]=1
    else:
        preds[i]=-1
    preds[i]=int(preds[i])

Save submit to csv file

In [ ]:
pred = preds
result=pd.DataFrame({'Event':[],'Q2':[]})
result['Event']=ev
result['Q2']=pred
result.Q2.replace({-1:'-1',
                   1:'1'}, inplace=True)

result.to_csv('final.csv',index=False)